In [2]:
import pandas as pd
import datetime as dt
import holidays as hld

#CABEÇALHO
print("=" * 72)
print(f'-------Automação para atualização de metas de venda e ativação----------')
print("=" * 72)
print(f'==== Ao inserir o nome das regionais, não coloque acentos =============')
print()

# Definindo as colunas
ano = 2024
Data = pd.date_range(start= f"{ano}-01-01", end=f"{ano}-12-31").repeat(14)
feriado_br = hld.Brazil(years=2024)  # Definindo feriados do Brasil em 2024
regionais = ["CAICO", "CAMPINA GRANDE","CAPIM GROSSO","FILADELFIA","GUARABIRA","JACOBINA","LAGOA SECA",
"LAJEDO","MONTEIRO","NATAL","PICUI","SANTA LUZIA","SENHOR DO BONFIM","SJ DO RIO DO PEIXE"] #14 REGIONAIS

# ESPECIFICIDADES DAS COLUNAS
arq_metas = pd.DataFrame(Data, columns=["Data"])
arq_metas['Data'] = arq_metas['Data'].dt.strftime('%Y-%m-%d')
arq_metas['Dia_semana'] = pd.to_datetime(Data).day_name(locale='pt_BR')
arq_metas['Mês'] = pd.to_datetime(Data).month_name(locale='pt_BR').str.upper()
arq_metas['Regional'] = [regionais[i % len(regionais)] for i in range(len(arq_metas))]

# Inicializando as colunas de 'Qtd ativação empresa', 'Qtd ativação comercial' e 'Qtd ativação operações' com 0
arq_metas['Qtd ativação empresa'] = 0
arq_metas['Qtd ativação comercial'] = 0
arq_metas['Qtd ativação operações'] = 0

# Marcar os dias de feriado na coluna 'Dia_semana'
for index, row in arq_metas.iterrows():
    data_str = row['Data']  # Pegando a data como string
    data_dt = dt.datetime.strptime(data_str, '%Y-%m-%d')  # Convertendo a string para datetime
    if data_dt in feriado_br:
        arq_metas.at[index, 'Dia_semana'] = 'Feriado'

# Função genérica para cadastro de valores
def cadastro_valores(nome_coluna, regra_distribuicao):
    while True:
        # Input do mês
        mes_esp = str(input(f'Informe o mês para atualização da coluna {nome_coluna}: (ou digite "fim" para encerrar) ')).upper()
        if mes_esp == 'FIM':
            print(f'A atualização da coluna {nome_coluna} foi finalizada.')
            break
        
        # Input da regional
        regional_esp = str(input('Informe a regional para atualização: ')).upper()
        
        # Input do número de ativações
        num_atv = float(input(f'Informe a quantidade de ativação para a regional {regional_esp} no mês {mes_esp}: '))
        
        # Filtro para encontrar as linhas com o mês e a regional especificados
        filtro = (
            (arq_metas['Mês'].str.upper() == mes_esp.upper()) & 
            (arq_metas['Regional'] == regional_esp)
        )
        
        # Confirmação dos valores
        print(f'Você inseriu o valor de {num_atv} na coluna {nome_coluna} para a regional {regional_esp}. Este valor está correto?')
        confirmacao = input('Os valores estão corretos? (s/n): ').lower()

        if confirmacao == 's':
            # Convertendo as colunas para o tipo float
            arq_metas['Qtd ativação empresa'] = arq_metas['Qtd ativação empresa'].astype(float)
            arq_metas['Qtd ativação comercial'] = arq_metas['Qtd ativação comercial'].astype(float)
            arq_metas['Qtd ativação operações'] = arq_metas['Qtd ativação operações'].astype(float)
            # Aplicar a regra de distribuição fornecida para cada coluna
            regra_distribuicao(filtro, num_atv, nome_coluna)
            print(f'O valor foi salvo com sucesso na coluna {nome_coluna}!')
        else:
            print("Corrija o valor abaixo:\n")
            continue

# Regras de distribuição específicas para cada coluna
def distribuicao_empresa(filtro, valor, coluna):
    # Distribuir os valores para dias de segunda a sábado, exceto feriados
    arq_metas.loc[filtro & arq_metas['Dia_semana'].isin(['Segunda-feira', 
                                                         'Terça-feira', 
                                                         'Quarta-feira',
                                                         'Quinta-feira', 
                                                         'Sexta-feira']) & 
                  (arq_metas['Dia_semana'] != 'Feriado'),
                  coluna] = round(valor * 0.92, 3)
    
    arq_metas.loc[filtro & arq_metas['Dia_semana'].isin(['Sábado']) & 
                  (arq_metas['Dia_semana'] != 'Feriado'),
                  coluna] = round(valor * 0.08, 3)

def distribuicao_comercial(filtro, valor, coluna):
    # Semelhante à empresa: segunda a sábado, exceto feriados
    arq_metas.loc[filtro & arq_metas['Dia_semana'].isin(['Segunda-feira', 
                                                         'Terça-feira', 
                                                         'Quarta-feira',
                                                         'Quinta-feira', 
                                                         'Sexta-feira']) & 
                  (arq_metas['Dia_semana'] != 'Feriado'),
                  coluna] = round(valor * 0.92, 3)
    
    arq_metas.loc[filtro & arq_metas['Dia_semana'].isin(['Sábado']) & 
                  (arq_metas['Dia_semana'] != 'Feriado'),
                  coluna] = round(valor * 0.08, 3)


def distribuicao_operacoes(filtro, valor, coluna):
    # Operações ocorrem de segunda a sexta, exceto feriados
    arq_metas.loc[filtro & ~arq_metas['Dia_semana'].isin(['Sábado', 'Domingo', 'Feriado']), coluna] = valor

# Chamando a função para cada coluna
print(f"-----> Atualização da coluna 'Qtd ativação empresa'")
cadastro_valores('Qtd ativação empresa', distribuicao_empresa)

print(f"-----> Atualização da coluna 'Qtd ativação comercial'")
cadastro_valores('Qtd ativação comercial', distribuicao_comercial)

print(f"-----> Atualização da coluna 'Qtd ativação operações'")
cadastro_valores('Qtd ativação operações', distribuicao_operacoes)

# Salvar o arquivo final em Excel
arq_metas.to_excel("Planilha_Metas.xlsx", index=False)
print("Arquivo Excel criado com sucesso!")


-------Automação para atualização de metas de venda e ativação----------
==== Ao inserir o nome das regionais, não coloque acentos =============

-----> Atualização da coluna 'Qtd ativação empresa'


Informe o mês para atualização da coluna Qtd ativação empresa: (ou digite "fim" para encerrar)  novembro
Informe a regional para atualização:  natal
Informe a quantidade de ativação para a regional NATAL no mês NOVEMBRO:  30


Você inseriu o valor de 30.0 na coluna Qtd ativação empresa para a regional NATAL. Este valor está correto?


Os valores estão corretos? (s/n):  s


O valor foi salvo com sucesso na coluna Qtd ativação empresa!


...

A atualização da coluna Qtd ativação empresa foi finalizada.
-----> Atualização da coluna 'Qtd ativação comercial'


Informe o mês para atualização da coluna Qtd ativação comercial: (ou digite "fim" para encerrar)  fim


A atualização da coluna Qtd ativação comercial foi finalizada.
-----> Atualização da coluna 'Qtd ativação operações'


Informe o mês para atualização da coluna Qtd ativação operações: (ou digite "fim" para encerrar)  fim


A atualização da coluna Qtd ativação operações foi finalizada.
Arquivo Excel criado com sucesso!
